In [11]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage.feature import hog
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
%matplotlib inline


In [12]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [13]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,9,19,29,39,49,59,69,79,89,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female  male
age                   
(9, 19]     3870  4350
(19, 29]    5760  7590
(29, 39]    1530  2580
(39, 49]    2070  1890
(49, 59]    1590  1170
(59, 69]     420   450
(69, 79]     360    30


In [14]:
def label_fetch(age):
    if age<10:
        return ("0-9")
    elif age<20:
        return ("10-19")
    elif age<30:
        return ("20-29")
    elif age<40:
        return ("30-39")
    elif age<50:
        return ("40-49")
    elif age<60:
        return ("50-59")
    elif age<70:
        return ("60-69")
    elif age<80:
        return ("70-79")
    elif age<90:
        return ("80-89")
    elif age<100:
        return ("90-99")

In [15]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
male_age_label = []
female_age_label = []
male_hog_features = []
female_hog_features = []


with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #Resizing the images
        resized_img = resize(img, (128,64)) 
        #creating hog features for the data
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)

        #Set age range label after gender bias
        #appending the hog features for image onto the hog data lists 
        if gender[i]== 'male':
            male_hog_features.append(fd)
            male_age_label.append(label_fetch(age[i]))
        else:
            female_hog_features.append(fd)
            female_age_label.append(label_fetch(age[i]))
                            
    

In [16]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(male_age_label)
X =  np.array(male_hog_features)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

#Linear SVM
print("MALE")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

MALE
Overall Accuracy using Linear SVM: 0.7385906040268456
              precision    recall  f1-score   support

       10-19       0.81      0.58      0.68      1451
       20-29       0.67      0.92      0.78      2467
       30-39       0.83      0.52      0.64       882
       40-49       0.83      0.66      0.73       621
       50-59       0.81      0.74      0.77       379
       60-69       0.98      0.84      0.90       152
       70-79       1.00      0.88      0.93         8

    accuracy                           0.74      5960
   macro avg       0.85      0.73      0.78      5960
weighted avg       0.76      0.74      0.73      5960

Accuracy for each class
[0.58373535 0.92217268 0.51927438 0.65861514 0.73614776 0.83552632
 0.875     ]




Non-Linear RBF Accuracy: 0.4139261744966443
              precision    recall  f1-score   support

       10-19       0.00      0.00      0.00      1451
       20-29       0.41      1.00      0.59      2467
       30-39       0.00      

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Non-Linear Poly Accuracy: 0.8536912751677852
              precision    recall  f1-score   support

       10-19       0.85      0.81      0.83      1451
       20-29       0.84      0.90      0.87      2467
       30-39       0.87      0.78      0.83       882
       40-49       0.87      0.83      0.85       621
       50-59       0.87      0.88      0.88       379
       60-69       0.97      0.93      0.95       152
       70-79       1.00      1.00      1.00         8

    accuracy                           0.85      5960
   macro avg       0.90      0.88      0.89      5960
weighted avg       0.85      0.85      0.85      5960

              precision    recall  f1-score   support

       10-19       0.85      0.81      0.83      1451
       20-29       0.84      0.90      0.87      2467
       30-39       0.87      0.78      0.83       882
       40-49       0.87      0.83      0.85       621
       50-59       0.87      0.88      0.88       379
       60-69       0.97      0.93

Iteration 212, loss = 0.03592037
Iteration 213, loss = 0.03518919
Iteration 214, loss = 0.03427757
Iteration 215, loss = 0.03380548
Iteration 216, loss = 0.03279700
Iteration 217, loss = 0.03230946
Iteration 218, loss = 0.03153240
Iteration 219, loss = 0.03101707
Iteration 220, loss = 0.03035057
Iteration 221, loss = 0.02988581
Iteration 222, loss = 0.02917651
Iteration 223, loss = 0.02838855
Iteration 224, loss = 0.02803602
Iteration 225, loss = 0.02739342
Iteration 226, loss = 0.02692832
Iteration 227, loss = 0.02657342
Iteration 228, loss = 0.02593101
Iteration 229, loss = 0.02554585
Iteration 230, loss = 0.02504130
Iteration 231, loss = 0.02452261
Iteration 232, loss = 0.02411410
Iteration 233, loss = 0.02378130
Iteration 234, loss = 0.02331944
Iteration 235, loss = 0.02288399
Iteration 236, loss = 0.02240294
Iteration 237, loss = 0.02206110
Iteration 238, loss = 0.02168765
Iteration 239, loss = 0.02129748
Iteration 240, loss = 0.02092078
Iteration 241, loss = 0.02064946
Iteration 

Iteration 461, loss = 0.00315937
Iteration 462, loss = 0.00314674
Iteration 463, loss = 0.00313149
Iteration 464, loss = 0.00311277
Iteration 465, loss = 0.00310311
Iteration 466, loss = 0.00308507
Iteration 467, loss = 0.00307776
Iteration 468, loss = 0.00304912
Iteration 469, loss = 0.00304256
Iteration 470, loss = 0.00303397
Iteration 471, loss = 0.00301086
Iteration 472, loss = 0.00300092
Iteration 473, loss = 0.00299524
Iteration 474, loss = 0.00297896
Iteration 475, loss = 0.00296205
Iteration 476, loss = 0.00295518
Iteration 477, loss = 0.00294149
Iteration 478, loss = 0.00292945
Iteration 479, loss = 0.00291725
Iteration 480, loss = 0.00289848
Iteration 481, loss = 0.00288626
Iteration 482, loss = 0.00287346
Iteration 483, loss = 0.00286517
Iteration 484, loss = 0.00286161
Iteration 485, loss = 0.00283972
Iteration 486, loss = 0.00282431
Iteration 487, loss = 0.00281187
Iteration 488, loss = 0.00280722
Iteration 489, loss = 0.00279693
Iteration 490, loss = 0.00278569
Iteration 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# Fit a SVM classifier to the data where gender=female
y =  np.array(female_age_label)
X = np.array(female_hog_features)   
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("FEMALE")
print("Overall Accuracy using Linear SVM:"+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0
    
#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
    
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')   
#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
    
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
    
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')


FEMALE
Overall Accuracy using Linear SVM:0.7558275058275058
              precision    recall  f1-score   support

       10-19       0.80      0.68      0.73      1251
       20-29       0.70      0.90      0.78      1910
       30-39       0.89      0.42      0.57       505
       40-49       0.79      0.74      0.77       709
       50-59       0.78      0.81      0.80       520
       60-69       1.00      0.53      0.70       146
       70-79       0.90      0.84      0.87       107

    accuracy                           0.76      5148
   macro avg       0.84      0.70      0.75      5148
weighted avg       0.77      0.76      0.75      5148

Accuracy for each class
[0.67705835 0.89790576 0.41980198 0.74330042 0.81153846 0.53424658
 0.8411215 ]




